# **Assignment 1**: Computing the area of the Mandelbrot Set

Karolina Chlopicka, 15716546 <br>
Shania Sinha, 14379031 <br>
Salomé Poulain, 13955993

In [ ]:
# Libraries
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Tuple, Callable
from scipy.stats import qmc  # For Latin Hypercube sampling
from joblib import Parallel, delayed

%matplotlib inline

## Problem 1: Mandelbrot Set

Creating and visualizing an example of the Mandelbrot set.

In [ ]:
BOUNDING_BOX_X = (-2.5, 1.5)
BOUNDING_BOX_Y = (-2, 2)
BOUND = 2
RESOLUTION_POINTS = 500
POWER = 2
MAX_ITER = 100 # Maximum number of iterations to check for convergence
SEED = 42

LITERARY_VALUE = 1.506591856 ####### idk

# COLOURS ??? maybe nice for the future 

In [ ]:
def build_mandelbrot(
    X: np.ndarray, 
    Y: np.ndarray, 
    bound: float = BOUND,
    power: int = POWER, 
    max_iter: int = MAX_ITER
) -> List[List[int]]:
    """
    Compute the Mandelbrot set for given x and y ranges.

    Parameters:
        X (np.ndarray): Array of x values (real parts of complex plane).
        Y (np.ndarray): Array of y values (imaginary parts of complex plane).
        bound (float): Threshold for divergence in the Mandelbrot set.
        power (int): The exponent for complex power in Mandelbrot iteration.
        max_iter (int): Maximum number of iterations to compute per point.

    Returns:
        List[List[int]]: 2D list where each element represents the iteration count 
                         before divergence, or 0 if the point belongs to the set.
    """
    mandelbrot_set = []
    X, Y = np.meshgrid(X, Y)
    C = X + 1j * Y  # Create a grid of complex numbers
    Z = np.zeros_like(C, dtype=complex)
    div_iter = np.zeros(C.shape, dtype=int)  # Track divergence iteration counts

    # Iteratively compute the Mandelbrot set
    for i in range(1, max_iter + 1):
        mask = np.abs(Z) < bound
        Z[mask] = Z[mask] ** power + C[mask]  # Update only non-diverging points
        div_iter[mask & (np.abs(Z) >= bound)] = i  # Record when they diverge

    mandelbrot_set = div_iter.tolist()  # Convert to a list of lists
    return mandelbrot_set

In [ ]:
# Setting parameters
all_x = np.linspace(BOUNDING_BOX_X[0], BOUNDING_BOX_X[1], RESOLUTION_POINTS)
all_y = np.linspace(BOUNDING_BOX_Y[0], BOUNDING_BOX_Y[1], RESOLUTION_POINTS)

colormap = 'magma'

mandelbrot_set = build_mandelbrot(all_x, all_y)

In [ ]:
# Plotting
ax = plt.axes()
ax.set_aspect('equal')
graph = ax.pcolormesh(all_x, all_y, mandelbrot_set, cmap = colormap, shading='auto')
plt.colorbar(graph)
plt.xlabel("Real-Axis")
plt.ylabel("Imaginary-Axis")
plt.title(f"Mandelbrot set for $z_{{new}} = z^{{power}} + c$")
plt.show()

## Problem 2: Investigate the convergence of $A_{i,s} \rightarrow A_M$

Investigating the area of the Mandelbrot set $A_M$ using Monte Carlo integration. $A_{i,s}$ denotes an estimate of an area, where $i$ refers to a number of iterations and $s$ refers to number of samples drawn.  

In [ ]:
# Function to check if a point is in the Mandelbrot set
def check_mandelbrot(x,y):
    c = complex(x,y)
    z = 0
    for j in range(100):
        if(abs(z) >= bound):
            answer = 0
            break
        else: z = z**2 + c
    else:
        answer = 1
    return(answer)

In [ ]:
# Monte Carlo simulation
def monte_carlo(i,s):
    iterations = i  # Number of iterations
    sample_size = s # Number of samples 
    size_iterations = np.zeros(iterations)

    for j in range(iterations):
        
        point_in = np.zeros(sample_size)    # Storing points inside the Mandelbrot set
        point_out = np.zeros(sample_size)   # Storing points outside the Mandelbrot set
        
        for i in range(sample_size):
        # Generating number of point equivalent to the sample size within the bounding rectangle
        # X in (-2,1.5) and Y in (-2,2)
            x_cordinate = np.random.uniform(-2,1.5) 
            y_cordinate = np.random.uniform(-2,2)

            if(check_mandelbrot(x_cordinate,y_cordinate)==1):
                point_in[i] = 1
            
            else: 
                point_out[i] = 1

        size = (np.count_nonzero(point_in == 1)/sample_size)*(3.5*4)
        size_iterations[j] = size   
    
    average_size = np.mean(size_iterations)  
    
    # return(f"For {i} iternations and sample size equal to {s} $A_i_s$ = {average_size}")
    return(average_size)

In [ ]:
#  Testing the function
monte_carlo(100,1000)

### 2.1: Convergence of $A_{i,s} \rightarrow A_M$ 

We set the sample size to $s=1000$, and investigate the convergence of the area when changing the number of iterations for all $j<i$ such that $i = 3000$ and $j = \{100k: k \in \{1, 2, ..., 30\}\}$

In [ ]:
number_of_iterations = np.array([100*i for i in range(1,31)])
estimation = np.zeros(len(number_of_iterations)) # Store the results of estimated area 

for i in range(len(number_of_iterations)):   
    iterations = number_of_iterations[i]   
    estimation[i] = monte_carlo(iterations,1000)
print(estimation)  

In [ ]:
figure_1, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

# Plotting the estimated area of the Mandelbrot set
ax1.plot(number_of_iterations,estimation, color = "blue")
ax1.set_xlabel("Number of iterations")
ax1.set_ylabel("Estimated area of Mandelbrot set")
ax1.grid(True)
ax2.grid(True)

# Plotting the difference in the estimated area
max_i = estimation[-1]
difference = np.abs(estimation[0:-1] - max_i)
ax2.plot(number_of_iterations[0:-1],difference, color = "blue")
ax2.set_xlabel("Number of iterations")
ax2.set_ylabel("Difference in the estimated area: $\parallel A(j,1000) - A(3000,1000) \parallel$")

## Problem 3: Different Sampling Methods

Explore different sampling methods. Namely:
1. Pure random sampling
2. Latin Hypercube sampling
3. Orthogonal sampling

### 3.1: Pure Random

In [ ]:
# TODO

### 3.2: Latin Hypercube

In [ ]:
# TODO

### 3.3: Orthogonal

In [ ]:
from scipy.stats import qmc

def oa_lhs(p, d, seed=None):
    """Generates an orthogonal array Latin hypercube sample.

    Author: Pamphile Tupui ROY
    Source: https://gist.github.com/tupui/3b79ecea1631e8925f3d47069d435b0f

    Inputs:
        p: number of levels
        d: number of dimensions
        seed: random seed

    Output: orthogonal array Latin hypercube sample
    """
    oa_sample = np.zeros(shape=(p**2, p + 1))

    arrays = np.tile(np.arange(p), (2, 1))
    oa_sample[:, :2] = np.stack(np.meshgrid(*arrays), axis=-1).reshape(-1, 2)
    for p_ in range(1, p):
        oa_sample[:, 2 + p_ - 1] = np.mod(oa_sample[:, 0] + p_ * oa_sample[:, 1], p)

    # scramble the OA
    oa_sample_ = np.empty(shape=(p**2, p + 1))
    for j in range(p + 1):
        perms = np.random.permutation(p)
        for k in range(p):
            idx = np.where(oa_sample[:, j] == k)[0]
            oa_sample_[idx, j] = perms[k]
    
    oa_sample = oa_sample_

    # Convert to OA-LHS
    oa_lhs_sample = np.zeros(shape=(p**2, p + 1))
    for j in range(p + 1):
        for k in range(p):
            idx = np.where(oa_sample[:, j] == k)[0]
            lhs = qmc.LatinHypercube(d=1, centered=True, seed=seed).random(p).flatten()
            oa_lhs_sample[:, j][idx] = lhs + oa_sample[:, j][idx]

    oa_lhs_sample /= p

    if d is not None:
        oa_lhs_sample = oa_lhs_sample[:, :d]
    return oa_lhs_sample

In [ ]:
def monte_carlo_oa_lhs(iterations, sample_size, x_min=-2, x_max=1.5, y_min=-2, y_max=2, seed=None):
    # Calculate p to ensure p^2 >= sample_size
    p = int(np.ceil(np.sqrt(sample_size)))
    size_iterations = np.zeros(iterations)

    for j in range(iterations):
        point_in = np.zeros(sample_size)    # Storing points inside the Mandelbrot set
        
        # Generate OA-LHS samples for the bounding rectangle
        oa_lhs_samples = oa_lhs(p, 2, seed=seed)
        
        # Scale the OA-LHS samples to the desired region
        oa_lhs_samples[:, 0] = oa_lhs_samples[:, 0] * (x_max - x_min) + x_min
        oa_lhs_samples[:, 1] = oa_lhs_samples[:, 1] * (y_max - y_min) + y_min

        # Limit the samples to sample_size
        oa_lhs_samples = oa_lhs_samples[:sample_size]
        
        for i in range(sample_size):
            x_cordinate, y_cordinate = oa_lhs_samples[i]
            if check_mandelbrot(x_cordinate, y_cordinate) == 1:
                point_in[i] = 1
        
        # Calculate the estimated area for this iteration
        size = (np.count_nonzero(point_in == 1) / sample_size) * ((x_max - x_min) * (y_max - y_min))
        size_iterations[j] = size   
    
    # Calculate the average estimated area across all iterations
    average_size = np.mean(size_iterations)
    
    return average_size

In [ ]:
# Example usage
average_size = monte_carlo_oa_lhs(iterations=100, sample_size=10000, seed=42)
print(f"Estimated area of the Mandelbrot set: {average_size}")

## Problem 4: Improvements to Monte Carlo Convergence Rate

Formulate and test a method to further improve the convergence rate of the Monte
Carlo approach.